In [ ]:
import math

def text_to_mem(input_file, output_mem_file):
    with open(input_file, "r") as fin, open(output_mem_file, "w") as fout:
        for line in fin:
            line = line.strip()
            if not line:
                continue

            # Convert symbols to bits
            bits = ''.join('1' if ch == '@' else '0' for ch in line)

            # Pad to nearest multiple of 32
            curr_len = len(bits)
            padded_len = math.ceil(curr_len / 32) * 32
            bits = bits.ljust(padded_len, '0')

            # Write to .mem (binary)
            fout.write(bits + "\n")

text_to_mem("inp.txt", "padded_inp.mem")

In [1]:
import numpy as np

# -------------------------
# Step 1: Read input file
# -------------------------
def read_input(filename):
    rows = []
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            row = [1 if c == '@' else 0 for c in line]
            rows.append(row)
    return rows


# -------------------------
# Step 2: Pad rows to 160 bits
# -------------------------
def pad_rows(rows, target_len=160):
    padded = []
    for row in rows:
        if len(row) > target_len:
            raise ValueError("Row longer than target length")
        padded.append(row + [0] * (target_len - len(row)))
    return np.array(padded, dtype=np.uint8)


# -------------------------
# Step 3: One-layer erosion
# -------------------------
def erode_once(mat):
    h, w = mat.shape
    out = np.zeros_like(mat)

    for i in range(h):
        for j in range(w):
            if mat[i, j] == 0:
                continue

            cnt = 0
            for di in (-1, 0, 1):
                for dj in (-1, 0, 1):
                    if di == 0 and dj == 0:
                        continue
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < w:
                        cnt += mat[ni, nj]

            if cnt >= 4:
                out[i, j] = 1

    return out

def bits_to_hex(bits):
    # Pad to multiple of 4 (34 → 36 bits)
    if len(bits) % 4 != 0:
        bits = [0] * (4 - len(bits) % 4) + bits

    hex_chars = []
    for i in range(0, len(bits), 4):
        nibble = bits[i:i+4]
        v = (nibble[0] << 3) | (nibble[1] << 2) | (nibble[2] << 1) | nibble[3]
        hex_chars.append(format(v, 'X'))

    return ''.join(hex_chars)

# -------------------------
# Step 4: 34-bit sliding window → hex
# -------------------------
def print_chunked_hex(mat):
    a = []
    h, w = mat.shape
    assert w % 32 == 0

    for r in range(h):
        row = mat[r].tolist()

        for s in range(0, w, 32):   # 👈 step = 32, not 1
            prev_bit = row[s - 1] if s > 0 else 0
            next_bit = row[s + 32] if s + 32 < w else 0

            bits = [prev_bit] + row[s:s + 32] + [next_bit]

            print(bits_to_hex(bits))
            a.append(str(bits_to_hex(bits)))
    return a


# -------------------------
# Main
# -------------------------
rows = read_input("inp.txt")
padded = pad_rows(rows, 160)
eroded = erode_once(padded)
a = print_chunked_hex(eroded)


018D8C843
380107180
022300460
018011901
1E0000000
00DF9CCF7
34C08F98F
375FA0AF4
0BC41BE8B
3F0000000
00F01FEEB
3F000B207
3E3E703FC
0F0C97906
2FD800000
1EEE3F7CF
3EC075DC3
3D16106BF
3F08F80E7
3DF000000
1B9F7FF8E
2DA07DBE3
37311C7FB
3F10FFCF2
2E4800000
1F33DDDCB
36E5F61F5
1C799824F
3EEB2B6B1
1C1000000
07E275FA5
1DCFEFDFF
3836CC1E7
374E1F1AB
35B800000
0EE7DCF81
1389FFFCF
381FCE1C3
358EAA0FF
3FD800000
1CEDC65C3
3710EEDE6
281F940E6
2EEDEFDEB
37F800000
1D77803E7
3D99047FC
011EF05EA
267BD74FE
2F6000000
0FFD68ED9
188B98FEE
2D8AFAE7F
3FFB7F8DE
2BE000000
02D8D07ED
1CC3D7BC5
1D976F43F
3BF41D8EF
37B800000
072069EF3
3EDB8D94E
293FFDC37
307F3343B
3DC800000
0EF8FFF61
1CEE95B88
0079FC67A
20EBFFE3E
2ED800000
0F9CD9081
18591B58E
22B7686DC
04D0EBFCF
33F000000
0FF900F01
10EE1EC6F
37CBACB62
2F01D85F1
1BA000000
075F00FE0
01A51F61F
3FF0D4E27
3E0638DF8
0ED000000
086E055D0
03FB1079B
3A35A041F
3B1F1CE3C
05F800000
103B2FE60
037C18F9C
017FD0058
049B39C03
3F7800000
1F9F5FDE4
0DDF97D56
22F8BC8E0
099FF8207
3FDC00000


In [52]:
import pandas as pd
data = pd.read_csv(r"C:\Users\Vatsal Melwani\EE\FPGA\Advent_of_Code_JS\Puzzle 4\part_2\part_2.sim\sim_1\behav\xsim\out_data.csv")

In [58]:
b = np.array(data.iloc[26:, 0], dtype = str)
a = np.array(a, dtype = str)
a = np.char.lower(a)
c = a!=b[:695]
print(np.sum(c))
print(a[382], b[382])

0
3ed35947a 3ed35947a


In [51]:
print(len(a))

695


In [16]:
import numpy as np

# ----------------------------
# INPUT (replace with real data)
# ----------------------------
M = rows

# Right-pad zeros to make 160 columns
M = np.pad(M, ((0, 0), (0, 21)), mode='constant')

ROWS, COLS = M.shape
assert COLS == 160

total_removed = 0


# ----------------------------
# helper: 32 bits -> hex
# ----------------------------
def bits_to_hex(bits):
    val = 0
    for b in bits:
        val = (val << 1) | int(b)
    return f"{val:08X}"


# ----------------------------
# MAIN PROCESSING
# ----------------------------
for r in range(ROWS):

    row_above = M[r - 1] if r > 0 else np.zeros(160, dtype=int)
    row_curr  = M[r]
    row_below = M[r + 1] if r < ROWS - 1 else np.zeros(160, dtype=int)

    for c0 in range(0, 160, 32):

        # ---- FREEZE neighbourhood for this chunk ----
        # 34 columns: left pad + 32 chunk bits + right neighbour
        def get(row, idx):
            return row[idx] if 0 <= idx < 160 else 0

        win_above = [get(row_above, i) for i in range(c0 - 1, c0 + 33)]
        win_curr  = [get(row_curr,  i) for i in range(c0 - 1, c0 + 33)]
        win_below = [get(row_below, i) for i in range(c0 - 1, c0 + 33)]

        # ---- DECISION PHASE (NO MODIFICATION) ----
        to_zero = []   # indices (0..31) inside this chunk

        for i in range(1, 33):   # center bits only
            if win_curr[i] == 1:
                neighbours = (
                    sum(win_above[i - 1:i + 2]) +
                    sum(win_curr [i - 1:i + 2]) +
                    sum(win_below[i - 1:i + 2]) - 1
                )

                if neighbours < 4:
                    to_zero.append(i - 1)   # chunk-local index

        # ---- APPLY UPDATES (IN-PLACE, AFTER CHUNK) ----
        for idx in to_zero:
            row_curr[c0 + idx] = 0

        total_removed += len(to_zero)

        # ---- PRINT AFTER EACH CHUNK ----
        chunk_bits = row_curr[c0:c0 + 32]
        print(
            f"row={r:03d}, chunk={c0//32}: "
            f"hex=0x{bits_to_hex(chunk_bits)}, "
            f"total_removed={total_removed}"
        )


row=000, chunk=0: hex=0x0C6C6421, total_removed=12
row=000, chunk=1: hex=0xC00838C0, total_removed=24
row=000, chunk=2: hex=0x11180230, total_removed=37
row=000, chunk=3: hex=0x0C008C80, total_removed=50
row=000, chunk=4: hex=0xF0000000, total_removed=53
row=001, chunk=0: hex=0x06FCE673, total_removed=56
row=001, chunk=1: hex=0xA0007883, total_removed=66
row=001, chunk=2: hex=0xB8F80078, total_removed=74
row=001, chunk=3: hex=0x1C00DF01, total_removed=83
row=001, chunk=4: hex=0xF8000000, total_removed=85
row=002, chunk=0: hex=0x0780FF71, total_removed=88
row=002, chunk=1: hex=0xF0005803, total_removed=97
row=002, chunk=2: hex=0xF1F000FE, total_removed=103
row=002, chunk=3: hex=0x7800BC03, total_removed=111
row=002, chunk=4: hex=0x7E000000, total_removed=113
row=003, chunk=0: hex=0x6771FBE3, total_removed=118
row=003, chunk=1: hex=0xF003ACE1, total_removed=122
row=003, chunk=2: hex=0xE8B0015F, total_removed=129
row=003, chunk=3: hex=0xF807C033, total_removed=133
row=003, chunk=4: hex=0x

In [5]:
print(np.array(rows).shape)

(139, 139)


In [13]:
for i in range(2):
    print((rows[i][:33]))

[1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]
[0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]


In [33]:
print(rows[0][127:])
print(rows[1][127:])

[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0]


In [25]:
from collections import deque
import numpy as np


def read_grid_from_file(filename):
    with open(filename, "r") as f:
        grid = [
            [int(ch) for ch in line.strip()]
            for line in f
            if line.strip()
        ]
    return grid


def prune_8_and_count(grid):
    m, n = len(grid), len(grid[0])
    print("Grid size:", m, n)

    cnt = [[0] * n for _ in range(m)]
    q = deque()

    dirs = [
        (-1, -1), (-1, 0), (-1, 1),
        ( 0, -1),           ( 0, 1),
        ( 1, -1), ( 1, 0),  ( 1, 1)
    ]

    # Initial neighbor counts
    for i in range(m):
        for j in range(n):
            if grid[i][j] == 1:
                c = 0
                for dx, dy in dirs:
                    ni, nj = i + dx, j + dy
                    if 0 <= ni < m and 0 <= nj < n and grid[ni][nj] == 1:
                        c += 1
                cnt[i][j] = c
                if c < 4:
                    q.append((i, j))

    removed = 0

    # Erosion loop
    while q:
        i, j = q.popleft()

        if grid[i][j] == 0:
            continue

        grid[i][j] = 0
        removed += 1

        for dx, dy in dirs:
            ni, nj = i + dx, j + dy
            if 0 <= ni < m and 0 <= nj < n and grid[ni][nj] == 1:
                cnt[ni][nj] -= 1
                if cnt[ni][nj] == 3:
                    q.append((ni, nj))

    return removed, grid


# --------------------------------------------------
# Usage
# --------------------------------------------------
grid = read_grid_from_file("padded_output.txt")

removed, final_grid = prune_8_and_count(grid)

def print_grid_34bit_hex_chunks(grid):
    py_mem = []
    for row in grid:
        n = len(row)
        idx = 0
        row_hex_words = []

        while idx < n:
            # Extract 32 data bits
            data_bits = row[idx:idx+32]
            if len(data_bits) < 32:
                data_bits = data_bits + [0] * (32 - len(data_bits))

            # Left neighbor bit
            left_bit = row[idx - 1] if idx - 1 >= 0 else 0

            # Right neighbor bit
            right_idx = idx + 32
            right_bit = row[right_idx] if right_idx < n else 0

            # Construct 34-bit word
            bits_34 = [left_bit] + data_bits + [right_bit]

            # Convert to integer
            value = 0
            for b in bits_34:
                value = (value << 1) | b

            # 34 bits → needs 9 hex digits (ceil(34/4) = 9)
            row_hex_words.append(f"{value:09X}")
            py_mem.append(f"{value:09X}")

            idx += 32

        # Print one row per line
        print(" ".join(row_hex_words))
    return py_mem

print("Removed count:", removed)
print("Final grid shape:", np.array(final_grid).shape)
py_mem = np.char.lower(print_grid_34bit_hex_chunks(final_grid))

Grid size: 139 160
Removed count: 9122
Final grid shape: (139, 160)
000000000 000000000 000000000 000000001 1C0000000
00000CC00 000000000 000000000 000000003 3E0000000
00001FE00 000000000 00000000C 0E0000006 2E0000000
000E3F700 0000300C0 00000001F 3F0000007 3C0000000
001F7FF80 0000781E0 00000001B 3F0000002 2C0000000
0333DDD80 0000F61E0 00000000F 3E0000001 1C0000000
07E261F80 0001EFDE0 000600007 370000003 358000000
0EE7C0F00 0001FFFC0 000F00003 358000007 3FC000000
1CEDC0000 0000EEDE0 001F80000 0EE00006B 37E000000
1D7780000 0000007E0 001EF0000 0670000FE 2F6000000
0FFC00000 0000007E0 000AF8000 07F8000DE 2BC000000
02D800000 0000003C0 00076C000 03F40006F 378000000
070000000 000000000 0003FC000 007F0003B 3DC000000
0E0000000 000000000 0001F8000 0003C001E 2EC000000
0F0000000 000000000 000060000 0000E000F 33C000000
0F0000000 000000000 000000000 000050001 1B8000000
060000000 018000000 000000000 000638000 0EC000000
000000000 03F800000 003000000 000F18000 05F000000
000000000 037C00000 007800000 00

In [27]:
with open("mem_dump.hex") as f:
    mem_hex = [line.strip() for line in f if line.strip()]

for i in range(139):
    print(mem_hex[i*5] == py_mem[i*5], mem_hex[i*5+1] == py_mem[i*5+1], mem_hex[i*5+2] == py_mem[i*5+2], mem_hex[i*5+3] == py_mem[i*5+3], mem_hex[i*5+4] == py_mem[i*5+4])

True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True
True True True True True


In [28]:
for i in range(139):
    print(mem_hex[i*5], mem_hex[i*5+1], mem_hex[i*5+2], mem_hex[i*5+3], mem_hex[i*5+4])

000000000 000000000 000000000 000000001 1c0000000
00000cc00 000000000 000000000 000000003 3e0000000
00001fe00 000000000 00000000c 0e0000006 2e0000000
000e3f700 0000300c0 00000001f 3f0000007 3c0000000
001f7ff80 0000781e0 00000001b 3f0000002 2c0000000
0333ddd80 0000f61e0 00000000f 3e0000001 1c0000000
07e261f80 0001efde0 000600007 370000003 358000000
0ee7c0f00 0001fffc0 000f00003 358000007 3fc000000
1cedc0000 0000eede0 001f80000 0ee00006b 37e000000
1d7780000 0000007e0 001ef0000 0670000fe 2f6000000
0ffc00000 0000007e0 000af8000 07f8000de 2bc000000
02d800000 0000003c0 00076c000 03f40006f 378000000
070000000 000000000 0003fc000 007f0003b 3dc000000
0e0000000 000000000 0001f8000 0003c001e 2ec000000
0f0000000 000000000 000060000 0000e000f 33c000000
0f0000000 000000000 000000000 000050001 1b8000000
060000000 018000000 000000000 000638000 0ec000000
000000000 03f800000 003000000 000f18000 05f000000
000000000 037c00000 007800000 001b38000 0f7800000
0f8000000 01df00000 00f800000 001ff8000 0fdc00000


In [29]:
for i in range(139):
    print(py_mem[i*5], py_mem[i*5+1], py_mem[i*5+2], py_mem[i*5+3], py_mem[i*5+4])

000000000 000000000 000000000 000000001 1c0000000
00000cc00 000000000 000000000 000000003 3e0000000
00001fe00 000000000 00000000c 0e0000006 2e0000000
000e3f700 0000300c0 00000001f 3f0000007 3c0000000
001f7ff80 0000781e0 00000001b 3f0000002 2c0000000
0333ddd80 0000f61e0 00000000f 3e0000001 1c0000000
07e261f80 0001efde0 000600007 370000003 358000000
0ee7c0f00 0001fffc0 000f00003 358000007 3fc000000
1cedc0000 0000eede0 001f80000 0ee00006b 37e000000
1d7780000 0000007e0 001ef0000 0670000fe 2f6000000
0ffc00000 0000007e0 000af8000 07f8000de 2bc000000
02d800000 0000003c0 00076c000 03f40006f 378000000
070000000 000000000 0003fc000 007f0003b 3dc000000
0e0000000 000000000 0001f8000 0003c001e 2ec000000
0f0000000 000000000 000060000 0000e000f 33c000000
0f0000000 000000000 000000000 000050001 1b8000000
060000000 018000000 000000000 000638000 0ec000000
000000000 03f800000 003000000 000f18000 05f000000
000000000 037c00000 007800000 001b38000 0f7800000
0f8000000 01df00000 00f800000 001ff8000 0fdc00000


In [31]:
def hex_to_bin34(h):
    return format(int(h, 16), '034b')
for i in range(139):
    print(
        hex_to_bin34(mem_hex[i*5]),
        hex_to_bin34(mem_hex[i*5 + 1]),
        hex_to_bin34(mem_hex[i*5 + 2]),
        hex_to_bin34(mem_hex[i*5 + 3]),
        hex_to_bin34(mem_hex[i*5 + 4]),
    )


0000000000000000000000000000000000 0000000000000000000000000000000000 0000000000000000000000000000000000 0000000000000000000000000000000001 0111000000000000000000000000000000
0000000000000000001100110000000000 0000000000000000000000000000000000 0000000000000000000000000000000000 0000000000000000000000000000000011 1111100000000000000000000000000000
0000000000000000011111111000000000 0000000000000000000000000000000000 0000000000000000000000000000001100 0011100000000000000000000000000110 1011100000000000000000000000000000
0000000000111000111111011100000000 0000000000000000110000000011000000 0000000000000000000000000000011111 1111110000000000000000000000000111 1111000000000000000000000000000000
0000000001111101111111111110000000 0000000000000001111000000111100000 0000000000000000000000000000011011 1111110000000000000000000000000010 1011000000000000000000000000000000
0000110011001111011101110110000000 0000000000000011110110000111100000 0000000000000000000000000000001111 11111000000000000000